# Expanding on Ito et al. (2017) to recover simulated task activity and connectivity matrix


Set up environment

In [ ]:
import copy
import sys
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
import pandas as pd
import seaborn as sns
sns.set_style("white")
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARMA

## Import custom modules
# Primary module with most model functions
sys.path.append('../')
import model

# Module for FC regression
sys.path.append('../../utils/')
import multregressionconnectivity as mreg

Helper functions

In [ ]:
phi = lambda x: np.tanh(x)

def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)

def pshift(y,p):
    y_lag = np.roll(y, p)
    y_lag[:p] = 0
    return y_lag

## Neural network level


***Create structural and neural network***

One hub and two local networks

In [ ]:
# Parameters for subject's networks
ncommunities = 3
innetwork_dsity = .60
outnetwork_dsity = .08
hubnetwork_dsity = .25

nodespercommunity = 35
totalnodes = nodespercommunity*ncommunities

In [ ]:
# Construct structural matrix
W = model.generateStructuralNetwork(ncommunities=ncommunities,
                                    innetwork_dsity=innetwork_dsity,
                                    outnetwork_dsity=outnetwork_dsity,
                                    hubnetwork_dsity=hubnetwork_dsity,
                                    nodespercommunity=nodespercommunity,
                                    showplot=False)
# Construct synaptic matrix
G = model.generateSynapticNetwork(W, showplot=False)

### Rest activity

***Simulate resting state activity using information flow model***

Apply neural network model **(Eq 3 in paper)** to the connectivity matrix and calculate each nodes' activity for each time point

**Note that this is still in synaptic space (i.e. not convolved)**

In [ ]:
## Set up simulation parameters
#Tmaxrest = 60000 # 1min resting state data
Tmaxrest = 300000 # 5min resting state data
dt = 1.0
tau = 1.0
g = 1.0
s = 1.0

restdata = model.networkModel(G, 
                              Tmax=Tmaxrest,
                              dt=dt,
                              g=g,
                              s=s,
                              tau=tau,
                              I=None,
                              noise = 1)

In [ ]:
plt.figure()
plt.imshow(restdata, aspect='auto',origin='lower')
plt.title('Simulated resting-state activity', y=1.04, fontsize=16)
plt.ylabel('Regions',fontsize=14)
plt.xlabel('Time (s)',fontsize=14)
plt.xticks(np.arange(0,Tmaxrest+1,10000),np.arange(0,Tmaxrest+1,10000)/100)
plt.colorbar()

### Task activity 

- How do you decide the task's spatial specificity, i.e. which network and nodes within a network it affects primarily? 
    - In the simulations the approach is: For topdown tasks only the hub network is simulated. E.g. for task 1 the first quarter of nodes are stimulated, for task 2 the next quarter etc. 
    - For topdown and bottom up (where a bug was fixed) it is a bit more complicated. E.g. for task 5 the first quarter of the hub network and the nodes of network 1 (first local network) that connect to the hub network (network 0) are stimulated

***Hub network stimulation***

In [ ]:
Ci = np.repeat(np.arange(ncommunities),nodespercommunity) # Construct a community affiliation vector
hub_ind = np.where(Ci==0)[0] # Identify the regions associated with the hub network (hub network is by default the 0th network)

# Set number of time points for each task
#Tmax = 10000
Tmax = 300000 #increased to 5 minutes

T = np.arange(0,Tmax,dt)

taskdata = {}
stimtimes = {}
    
# Construct timing array for convolution -- this timing is irrespective of the task being performed
# Tasks are only determined by which nodes are stimulated
tasktiming = np.zeros((1,len(T)))
for t in range(len(T)):
    if t%2000>500 and t%2000<1000:
    #if t>0 and t%1000==0: #Changed to a task with spikes as activity
        tasktiming[0,t] = 1.0

In [ ]:
topdown_only = range(1,2)
task = 1

taskcount = task-np.min(topdown_only)
stimsize = np.floor(nodespercommunity/4.0)
stim_nodes = np.arange((taskcount)*stimsize,(taskcount)*stimsize+stimsize,dtype=int)
stimtimes[task] = np.zeros((totalnodes,len(T)))

for t in range(len(T)):
    if tasktiming[0,t] == 1:
        # Task stimulation every 10 seconds for 4 seconds, excluding the first 10 seconds
        # Changed to spike every second for 10 minues
        stimtimes[task][stim_nodes,t] = .5

Block design looks like:

In [ ]:
plt.plot(T[:10000], tasktiming[0,:10000])
plt.ylim(top = 1.2, bottom = -0.1)

Activated nodes in the hub network *only* (making it a top-down task) are:

In [ ]:
stim_nodes

Does **NOT** directly use restdata generated above. Uses the same parameters that were used for the rest data but the connectivity matrix should not be identical due to random noise added to spontaneous activity

In [ ]:
taskdata[task] = model.networkModel(G,Tmax=Tmax,dt=dt,g=g,s=s,tau=tau,
                                          I=stimtimes[task], noise=1)

## BOLD level


### Resting state


#### Convolve resting state activity

In [ ]:
TRLength = 100

restfmri = model.convolveTimeseries(restdata,
                                    samplingrate=dt,
                                    TRLength=TRLength)

In [ ]:
plt.figure()
plt.imshow(restfmri, aspect='auto',origin='lower')
plt.title('Simulated fMRI resting-state activity', y=1.04, fontsize=16)
plt.ylabel('Regions',fontsize=14)
plt.xlabel('Time (s)',fontsize=14)
plt.colorbar()

#### Recover connectivity matrix from resting state activity

In [ ]:
fcmat = np.corrcoef(restfmri)
# 0 out the diagonal
np.fill_diagonal(fcmat,0)

plt.figure()
plt.imshow(fcmat, origin='lower')
plt.colorbar()
plt.title('Simulated fMRI resting-state FC \nSingle Subject\nPearsonFC', y=1.04, fontsize=16)
plt.xlabel('Regions',fontsize=14)
plt.ylabel('Regions', fontsize=14)
plt.tight_layout()

In [ ]:
fcmat_mreg = mreg.multregressionconnectivity(restfmri)
# 0 out the diagonal
np.fill_diagonal(fcmat_mreg,0)

plt.figure()
plt.imshow(fcmat_mreg, origin='lower')
plt.colorbar()
plt.title('Simulated fMRI resting-state FC\nSingle Subject\nMultipleRegressionFC', y=1.04, fontsize=16)
plt.xlabel('Regions',fontsize=14)
plt.ylabel('Regions', fontsize=14)
plt.tight_layout()

How **quantitatively** similar is the fc_mat to the weight matrix G?

In [ ]:
plt.scatter(fcmat.flatten(), G.flatten())
np.corrcoef(fcmat.flatten(), G.flatten())

In [ ]:
plt.scatter(fcmat_mreg.flatten(), G.flatten())
np.corrcoef(fcmat_mreg.flatten(), G.flatten())

### Task fmri

##### Convolve task activity 


In [ ]:
samplingrate=1.0
# Convolve simulated task data into fMRI signal
task_bold = model.convolveTimeseries(taskdata[task],
                               samplingrate=samplingrate, 
                               TRLength=TRLength)


In [ ]:
plt.figure()
plt.imshow(task_bold, aspect='auto',origin='lower')
plt.title('Simulated single topdown task fMRI activity', y=1.04, fontsize=16)
plt.ylabel('Regions',fontsize=14)
plt.xlabel('Time (s)',fontsize=14)
plt.colorbar()

##### Run GLM on convolved task activity

In [ ]:
n_skip_BOLD = int(TRLength)
# Temporally downsample task timing to seconds (before convolving)
timingconv = tasktiming[:,::n_skip_BOLD]
timingconv.shape = (timingconv.shape[1],)
hrfsample_rate=1.0 # HRF is sampled at seconds
hrfsample_times = np.arange(0, 30, hrfsample_rate, dtype=float)
hrf_at_simsample = model.hrf(hrfsample_times)
hrfconvtime = np.convolve(timingconv, hrf_at_simsample)
n_to_remove = len(hrf_at_simsample) - 1
convolved = hrfconvtime[:-n_to_remove]
# Output
timing_convolved_downsampled = convolved

In [ ]:
task_betas_postfmri, task_resids_postfmri, task_tvals_postfmri, task_betas_ci_postfmri = model.runTaskGLM(task_bold, timing_convolved_downsampled)

Results of GLM. Vertical lines indicate network boundaries. Stimulated nodes are the first eight in the hub network.

In [ ]:
plt.plot(task_tvals_postfmri)
plt.ylabel('Uncorrected t-value',fontsize=14)
plt.xlabel('Node',fontsize=14)
plt.axhline(y=3,linewidth=2, color='red', ls = "--")
plt.axvline(x=35,linewidth=2, color='black', ls = "--")
plt.axvline(x=70,linewidth=2, color='black', ls = "--")

In [ ]:
uperror = task_betas_ci_postfmri[:,1]
downerror = task_betas_ci_postfmri[:,0]
errorbars = np.concatenate(([uperror], [downerror]), axis=0)

plt.errorbar(np.arange(totalnodes), task_betas_postfmri, yerr=errorbars, fmt = 'o')
plt.ylabel('Beta (with 95% CI)',fontsize=14)
plt.xlabel('Node',fontsize=14)

In [ ]:
hub_stim_nodes = stim_nodes

### Relationship between betas and weights from the connectivity matrix

Extract average connectivity between each node and the nodes stimulated by task.  
Correlate these average weights with betas for each node.

NOTE: The weight matrix is *NOT* symmetric (I think) because afferent and efferent connection weights need not be the same.

In [ ]:
check_symmetric(G)

Extract average connectivity of each node with the stim nodes

In [ ]:
afferent_cons = np.empty((0,G.shape[0]), float)
efferent_cons = np.empty((0,G.shape[0]), float)
for i in range(G.shape[0]):
    efferent_cons = np.append(efferent_cons, [G[hub_stim_nodes,i].mean()])
    afferent_cons = np.append(afferent_cons, [G[i,hub_stim_nodes].mean()])

In [ ]:
d = {'Beta': task_betas_postfmri[8:], 'Connectivity': efferent_cons[8:]}
df = pd.DataFrame(data=d)
p = sns.regplot(x='Beta', y='Connectivity', data = df)
p.text(min(task_betas_postfmri[8:]), max(efferent_cons[8:]), "r = %s"%(str(round(np.corrcoef(task_betas_postfmri[8:], efferent_cons[8:])[1][0],3))), horizontalalignment='left', color='black')
p.set_title('Correlation between betas and efferent connections')

In [ ]:
d = {'Beta': task_betas_postfmri[8:], 'Connectivity': afferent_cons[8:]}
df = pd.DataFrame(data=d)
p = sns.regplot(x='Beta', y='Connectivity', data = df)
p.text(min(task_betas_postfmri[8:]), max(afferent_cons[8:]), "r = %s"%(str(round(np.corrcoef(task_betas_postfmri[8:], afferent_cons[8:])[1][0],3))), horizontalalignment='left', color='black')
p.set_title('Correlation between betas and afferent connections')